In [41]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
import scipy.stats as st
from scipy.stats import linregress

In [42]:
# Output File (CSV)
pyre_data_path = "Resources/California_Fire_Incidents.csv"

In [43]:
pyre_data = pd.read_csv(pyre_data_path)
pyre_data.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,Started,Time.1,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,8/17/13,15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,5/30/13,15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,7/15/13,13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,8/10/13,16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,5/2/13,07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [44]:
pyre_df = pyre_data[["AcresBurned", "AirTankers", "ArchiveYear", "Counties", "CountyIds","CrewsInvolved", "Dozers", "Engines", "Extinguished", "Days Burned", "Fatalities", "Helicopters", "Injuries", "Latitude", "Location", "MajorIncident", "Name", "PersonnelInvolved", "Started", "StructuresDamaged", "StructuresDestroyed", "StructuresEvacuated", "StructuresThreatened", "UniqueId", "WaterTenders" 
]]
pyre_df.head()

,AcresBurned,AirTankers,ArchiveYear,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Days Burned,...,MajorIncident,Name,PersonnelInvolved,Started,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,WaterTenders
0,257314.0,NaN,2013,Tuolumne,55,NaN,NaN,NaN,9/6/13,20,...,False,Rim Fire,NaN,8/17/13,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,NaN
1,30274.0,NaN,2013,Los Angeles,19,NaN,NaN,NaN,6/8/13,9,...,False,Powerhouse Fire,NaN,5/30/13,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,NaN
2,27531.0,NaN,2013,Riverside,33,NaN,NaN,NaN,7/30/13,15,...,False,Mountain Fire,NaN,7/15/13,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,NaN
3,27440.0,NaN,2013,Placer,31,NaN,NaN,NaN,8/30/13,20,...,False,American Fire,NaN,8/10/13,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,NaN
4,24251.0,NaN,2013,Ventura,56,47.0,8.0,117.0,5/11/13,9,...,True,Springs Fire,2167.0,5/2/13,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,11.0


In [45]:
counties = pyre_df["Counties"].value_counts()
counties

Riverside          146
San Diego           89
Butte               66
San Luis Obispo     64
Shasta              64
Kern                62
Fresno              57
Siskiyou            57
San Bernardino      53
Tehama              51
Lake                49
Los Angeles         46
Monterey            45
Santa Clara         39
El Dorado           37
Lassen              36
Madera              36
Mariposa            35
Tulare              35
Alameda             32
Modoc               31
Ventura             30
Santa Barbara       29
Mendocino           28
Contra Costa        27
Napa                25
Humboldt            25
Tuolumne            22
Calaveras           22
Trinity             21
Stanislaus          20
San Benito          20
Solano              19
Sonoma              18
Placer              17
Nevada              17
Merced              16
Yuba                14
Amador              13
Yolo                12
Inyo                12
Sacramento          11
Plumas              11
Orange     

In [48]:
unique_id = pyre_df["UniqueId"].value_counts()
unique_id.head(30)

pyre_df = pyre_df.set_index('Name')
pyre_df.head()

,AcresBurned,AirTankers,ArchiveYear,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Days Burned,...,Location,MajorIncident,PersonnelInvolved,Started,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,WaterTenders
Name,,,,,,,,,,,,,,,,,,,,,
Rim Fire,257314.0,NaN,2013,Tuolumne,55,NaN,NaN,NaN,9/6/13,20,...,3 miles east of Groveland along Hwy 120,False,NaN,8/17/13,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,NaN
Powerhouse Fire,30274.0,NaN,2013,Los Angeles,19,NaN,NaN,NaN,6/8/13,9,...,Angeles National Forest,False,NaN,5/30/13,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,NaN
Mountain Fire,27531.0,NaN,2013,Riverside,33,NaN,NaN,NaN,7/30/13,15,...,Hwy 243 & Hwy 74 near Mountain Center,False,NaN,7/15/13,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,NaN
American Fire,27440.0,NaN,2013,Placer,31,NaN,NaN,NaN,8/30/13,20,...,"Deadwood Ridge, northeast of Foresthill",False,NaN,8/10/13,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,NaN
Springs Fire,24251.0,NaN,2013,Ventura,56,47.0,8.0,117.0,5/11/13,9,...,Southbound Highway 101 at Camarillo Springs Ro...,True,2167.0,5/2/13,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,11.0


In [32]:
unique_id2 = pyre_df.groupby(["UniqueId"]).count()["Name"]
unique_id2

UniqueId
00089805-cdee-4607-8b67-1653bee463bc    1
0011ce0f-5493-4bef-b6e1-4674a0393b0c    1
0089929d-0b52-46fe-8982-b096b7616f30    1
0089d3c4-c554-46e1-8eb9-af95e4dfae0c    1
00a13d84-415e-4106-aa31-2e664ed9eabf    1
                                       ..
ff8701f0-a12a-4b40-baa9-b58c4e2e31f3    1
ffb3293b-3349-49c5-a7cb-c488391e8a72    1
ffb57bcc-80f7-4299-bfcf-c9be3b66d5ff    1
ffbc1cb8-be33-4483-8442-85ab2587dce4    1
ffec404e-dea9-4175-8c50-8f2b0ca3efdd    1
Name: Name, Length: 1609, dtype: int64

In [52]:
major = pyre_df.loc[pyre_df["MajorIncident"]==True]
major.head()

major["MajorIncident"].count()

383

In [53]:
major.count()

AcresBurned             383
AirTankers               26
ArchiveYear             383
Counties                383
CountyIds               383
CrewsInvolved           167
Dozers                  119
Engines                 185
Extinguished            378
Days Burned             383
Fatalities               21
Helicopters              83
Injuries                119
Latitude                383
Location                383
MajorIncident           383
PersonnelInvolved       191
Started                 383
StructuresDamaged        66
StructuresDestroyed     160
StructuresEvacuated       0
StructuresThreatened     30
UniqueId                383
WaterTenders            141
dtype: int64

In [66]:
years = pyre_df.groupby(["ArchiveYear"]).count()["UniqueId"]
years

ArchiveYear
2013    162
2014    117
2015    152
2016    185
2017    438
2018    316
2019    266
Name: UniqueId, dtype: int64